**Quantitative Value Strategy**

In [217]:
import numpy as np
import pandas as pd
import yfinance as yf
import math
from scipy import stats
from statistics import mean

In [ ]:
stocks = pd.read_csv('sp_500_stocks_.csv')
stocks

In [219]:
my_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'PE Ratio', 
    'PE Ratio Percentile', 
    'PB Ratio', 
    'PB Ratio Percentile', 
    'PS Ratio', 
    'PS Ratio Percentile', 
    'EV/EBITDA', 
    'EV/EBITDA Percentile', 
    'EV/GP', 
    'EV/GP Percentile', 
    'RV Score'] 
final_dataframe = pd.DataFrame()
for ticker in stocks['Ticker']:
    stock = yf.Ticker(ticker)
    df = {
        'Ticker':ticker, 
        'Price': stock.info.get('currentPrice'), 
        'PE':stock.info.get('trailingPE'), 
        'PB':stock.info.get('priceToBook'), 
        'PS':stock.info.get('priceToSalesTrailing12Months'),
        'EV':stock.info.get('enterpriseValue'),
        'EBITDA':stock.info.get('ebitda'),
        'GP':stock.info.get('grossProfits') 
        }
    ticker_series = pd.Series(df['Ticker'])
    price_series = pd.Series(df['Price'])
    share_series = pd.Series(np.NaN)
    pe = pd.Series(df['PE'])
    pepercentile = pd.Series(np.NaN)
    pb = pd.Series(df['PB'])
    pbpercentile = pd.Series(np.NaN)
    ps = pd.Series(df['PS'])
    pspercentile = pd.Series(np.NaN)
    if df['EV'] is not None and df['EBITDA'] is not None:
        evEBITDA = pd.Series(df['EV'] / df['EBITDA'])
    else:
        evEBITDA=pd.Series(np.NaN)
    evEBITDApercentile = pd.Series(np.NaN)
    if df['EV'] is not None and df['GP'] is not None:
        evGP = pd.Series(df['EV'] / df['GP'])
    else:
        evGP = pd.Series(np.NaN)
    evGPpercentile = pd.Series(np.NaN)
    rv_score = pd.Series(np.NaN)
    data = pd.concat(
        [ticker_series, 
         price_series, 
         share_series, 
         pe, 
         pepercentile, 
         pb, 
         pbpercentile, 
         ps, 
         pspercentile, 
         evEBITDA, 
         evEBITDApercentile, 
         evGP, 
         evGPpercentile, 
         rv_score
         ], axis=1, ignore_index=True)
    data.columns = my_columns
    final_dataframe = pd.concat([final_dataframe, data], ignore_index=True)
    
    

In [220]:
metrics = ['PE Ratio', 'PB Ratio', 'PS Ratio', 'EV/EBITDA', 'EV/GP']
for column in metrics:
    final_dataframe[column] = pd.to_numeric(final_dataframe[column], errors='coerce')
    final_dataframe[column].fillna(final_dataframe[column].mean(), inplace=True)

In [ ]:
for row in final_dataframe.index:
    percentiles = []
    for metric in metrics:
        final_dataframe.loc[row, f'{metric} Percentile'] = stats.percentileofscore(final_dataframe[f'{metric}'], final_dataframe.loc[row, f'{metric}']) / 100
        percentiles.append(final_dataframe.loc[row, f'{metric} Percentile'])

    final_dataframe.loc[row, 'RV Score'] = mean(percentiles)

final_dataframe = final_dataframe.sort_values(by='RV Score', ascending=False, ignore_index=True)
final_dataframe = final_dataframe[:51]
final_dataframe


In [ ]:
portfolio_size = 10000000
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in final_dataframe.index:
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe